In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

import os

In [2]:
load_dotenv()

llm = ChatOpenAI( model="Qwen/Qwen2.5-7B-Instruct:together", api_key=os.environ["HF_TOKEN"], base_url="https://router.huggingface.co/v1" )

In [25]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    evaluation: int

In [7]:
def create_outline(state: BlogState) -> BlogState:

    #fetch title
    title = state['title']

    #call llm
    prompt = f"Create a detailed blog outline for the blog titled: {title}"
    outline = llm.invoke(prompt).content

    #update state
    state['outline'] = outline 
    return state

In [12]:
def create_blog(state: BlogState) -> BlogState:

    #fetch outline and title
    title = state['title']
    outline = state['outline']

    #call llm
    prompt = f"Write a detailed blog on the title: {title} with the following outline: {outline}"
    content = llm.invoke(prompt).content

    #update state
    state['content'] = content 
    return state

In [26]:
def evaluate_blog(state: BlogState) -> BlogState:

    #fetch outline and content
    outline = state['outline']
    content = state['content']

    #prompt
    prompt = f"Evaluate the following blog content based on the outline provided. Give a score out of 10.\n\nOutline: {outline}\n\nContent: {content}"
    evaluation = llm.invoke(prompt).content

    #update state
    state['evaluation'] = evaluation
    return state

In [27]:
graph = StateGraph(BlogState)

#adding nodes
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)
graph.add_node('evaluate_blog',evaluate_blog) 

#defining edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

#compile graph
workflow = graph.compile()

In [28]:
#execute the graph
initial_state={'title': "Is Vibe coding going to replace traditional programming in the next 5 years?"}

final_state =workflow.invoke(initial_state)

print(final_state)

{'title': 'Is Vibe coding going to replace traditional programming in the next 5 years?', 'outline': '### Blog Outline: Is Vibe Coding Going to Replace Traditional Programming in the Next 5 Years?\n\n#### Introduction\n- **Overview of Vibe Coding**: Brief introduction to Vibe coding, its core principles, and how it simplifies the coding process.\n- **Traditional Programming**: Brief overview of traditional programming languages and methodologies.\n- **Purpose of the Blog**: To explore the potential of Vibe coding and whether it could replace traditional programming in the next 5 years.\n\n#### What is Vibe Coding?\n- **Definition and Core Concepts**: Detailed explanation of what Vibe coding is, its key features, and how it differs from traditional programming.\n- **Advantages of Vibe Coding**:\n  - **Simplicity and Ease of Use**: How Vibe coding simplifies complex tasks and makes programming more accessible.\n  - **Faster Development**: How Vibe coding can lead to faster development cy

In [21]:
print(final_state['outline'])

### Blog Outline: Is Vibe Coding Going to Replace Traditional Programming in the Next 5 Years?

#### Introduction
- **Overview of Vibe Coding**: Brief introduction to Vibe coding, its core principles, and how it simplifies the coding process.
- **Traditional Programming**: Brief overview of traditional programming languages and methodologies.
- **Purpose of the Blog**: To explore the potential of Vibe coding and whether it could replace traditional programming within the next 5 years.

#### What is Vibe Coding?
- **Definition and Core Concepts**: Detailed explanation of what Vibe coding is, its key features, and how it differs from traditional programming.
- **Advantages of Vibe Coding**:
  - **Simplicity and Ease of Use**: How Vibe coding simplifies complex tasks and makes programming more accessible.
  - **Faster Development**: How Vibe coding can speed up the development process.
  - **Reduced Errors**: How it minimizes common coding errors and enhances code quality.
- **Use Cases**

In [22]:
print(final_state['content']) #final_state['content']

### Blog Outline: Is Vibe Coding Going to Replace Traditional Programming in the Next 5 Years?

#### Introduction
- **Overview of Vibe Coding**: Brief introduction to Vibe coding, its core principles, and how it simplifies the coding process.
- **Traditional Programming**: Brief overview of traditional programming languages and methodologies.
- **Purpose of the Blog**: To explore the potential of Vibe coding and whether it could replace traditional programming within the next 5 years.

#### What is Vibe Coding?
- **Definition and Core Concepts**: Detailed explanation of what Vibe coding is, its key features, and how it differs from traditional programming.
- **Advantages of Vibe Coding**:
  - **Simplicity and Ease of Use**: How Vibe coding simplifies complex tasks and makes programming more accessible.
  - **Faster Development**: How Vibe coding can speed up the development process.
  - **Reduced Errors**: How it minimizes common coding errors and enhances code quality.
- **Use Cases**

In [29]:
print(final_state['evaluation'])

Based on the provided outline and content, I will evaluate the blog content against the outlined structure and provide a score out of 10.

### Evaluation

#### Introduction
- **Overview of Vibe Coding**: The introduction provides a brief and clear overview of Vibe coding and its core principles.
- **Traditional Programming**: A brief overview of traditional programming is given, setting the stage for comparison.
- **Purpose of the Blog**: The purpose is clearly stated, aiming to explore the potential of Vibe coding and its likelihood of replacing traditional programming in the next 5 years.

**Score: 8/10** - The introduction is well-structured and provides a good foundation for the blog.

#### What is Vibe Coding?
- **Definition and Core Concepts**: The definition and core concepts are explained in detail.
- **Advantages of Vibe Coding**: The advantages are clearly outlined, including simplicity, faster development, and reduced errors.
- **Use Cases**: Examples of industries and appli